<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/AgenticAI/blob/main/Langchain%20Build%20LLM%20Application%20(Git.ir%20Course)/18_vectore_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cohere

In [1]:
!pip install -q langchain langchain_cohere cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.5/295.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install -q wikipedia langchain-chroma

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 889.6 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
os.environ["COHERE_API_KEY"] = "7L948hvhLOOJlmfgfEdk4bjxhATSHZ82vrxZZffJ"

from langchain_community.document_loaders import WikipediaLoader

wikipedia_docs = WikipediaLoader(query="The Lord of the Rings",
                                 load_max_docs=6,
                                 lang="en").load()
for doc in wikipedia_docs:
    doc.metadata.pop('summary', None)

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256, chunk_overlap=32
)

splitted_docs = recursive_splitter.split_documents(wikipedia_docs)

print('The number of splitted documents:', len(splitted_docs))

The number of splitted documents: 132


In [5]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-multilingual-light-v3.0",
    max_retries=5,
    request_timeout=20)


In [6]:
from langchain.vectorstores import Chroma

vector_store = Chroma.from_documents(
    documents=splitted_docs,
    embedding=embeddings,
    collection_name="the_lord_of_the_rings",
    persist_directory="./chroma_db"
)

print('The number of documents in the collection:', vector_store._collection.count())

The number of documents in the collection: 132


In [10]:
query = 'How many Golden Globe awards did the third Lord of the Rings film win?'

results = vector_store.similarity_search(query, k = 3)

for result in results:
    print('Document Title:', result.metadata['title'])
    print('Document Content:', result.page_content)
    print('-'*100)

Document Title: The Lord of the Rings (film series)
Document Content: The Lord of the Rings is a trilogy of epic fantasy adventure films directed by Peter Jackson, based on the novel The Lord of the Rings by English author J. R. R. Tolkien. The films are titled identically to the three volumes of the novel: The Fellowship of
----------------------------------------------------------------------------------------------------
Document Title: The Lord of the Rings
Document Content: Award-winning adaptations of The Lord of the Rings have been made for radio, theatre, and film. It was named Britain's best-loved novel of all time in a 2003 poll by the BBC called The Big Read.
----------------------------------------------------------------------------------------------------
Document Title: The Lord of the Rings (film series)
Document Content: The Lord of the Rings is widely regarded as one of the greatest and most influential film series ever made. It was a major financial success and is am

In [12]:
query = 'ارباب حلقه‌های سوم چند جایزه‌ی گلدن گلوب برنده شد؟'

results = vector_store.similarity_search_with_score(query, k = 3)

for result in results:
    doc = result[0]
    score = result[1]
    print('Score:', score)
    print('Document Title:', doc.metadata['title'])
    print('Document Content:', doc.page_content)
    print('-'*100)

Score: 1.2251390218734741
Document Title: The Lord of the Rings: The Fellowship of the Ring
Document Content: Awards, it was nominated for thirteen awards, including Best Picture, winning for Best Cinematography, Best Makeup, Best Original Score, and Best Visual Effects.
----------------------------------------------------------------------------------------------------
Score: 1.2715498208999634
Document Title: The Lord of the Rings: Gollum
Document Content: In the year 3017 of the Third Age, 66 years after Bilbo Baggins obtains the One Ring from Gollum in the Misty Mountains, Aragorn captures Gollum in the Dead Marshes and turns him over to Gandalf. Gollum had reached the confines of Mordor in 2980, meeting
----------------------------------------------------------------------------------------------------
Score: 1.2800406217575073
Document Title: The Lord of the Rings (1978 film)
Document Content: to go to Mordor, where the Ring can be destroyed. Thereafter Frodo sets off from Rivend